In [5]:
import base64
import requests
import json
from utils import *

In [12]:
# Function to encode the image to base64
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read()).decode()
    return encoded_string

def get_metadata_gpt(image_name):

    '''
    PROMPT TO GPT
    ------------------

    Based on the plot:

    1. What is the title of the x-axis?
    2. What is the range of the x-axis?
    3. What is the title of the y-axis?
    4. What is the range of the y-axis?
    5. What are the different types?

    Can you provide the answers to the aforementioned questions in the following format:

    answers = {
    "x-axis": {
    "title": [INSERT TITLE HERE IN STRING FORMAT],
    "range":  [INSERT RANGE HERE IN NUMERICAL FORMAT],
    },
    "y-axis": {
    "title": [INSERT TITLE HERE IN STRING FORMAT],
    "range":  [INSERT RANGE HERE IN NUMERICAL FORMAT],
    },
    "types": [INSERT THE TYPES IN LIST OF LISTS OF LENGTH = 2 WHERE EACH LIST LOOKS LIKE [TYPE NAME, MARKER COLOR in "black", "white", "red","green","blue", "yellow", "magenta", "cyan", "grey"]]
    }

    '''

    answers = None
    
    # Encode the image
    encoded_image = encode_image(image_name)

    headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {API_KEY}"
    }

    payload = {
    "model": "gpt-4-vision-preview",
    "messages": [
        {
        "role": "user",
        "content": [
            {
            "type": "text",
            "text": "Based on the plot: 1. Describe the title of the x-axis? 2. Describe the range of the x-axis? 3. Describe the title of the y-axis 4. Describe the range of the y-axis? 5. Describe the different types? Can you provide the answers to the aforementioned questions in the following format: answers = { \"x-axis\": { \"title\": [INSERT TITLE HERE IN STRING FORMAT], \"range\":  [INSERT RANGE HERE IN NUMERICAL FORMAT],}, \"y-axis\": { \"title\": [INSERT TITLE HERE IN STRING FORMAT], \"range\":  [INSERT RANGE HERE IN NUMERICAL FORMAT], }, \"types\": [INSERT THE TYPES IN LIST OF TUPLES OF LENGTH = 2 WHERE EACH TUPLE LOOKS LIKE (TYPE NAME, MARKER COLOR in \"black\", \"white\", \"red\",\"green\",\"blue\", \"yellow\", \"magenta\", \"cyan\", \"grey\")]}"
            },
            {
            "type": "image_url",
            "image_url": {
                "url": f"data:image/jpeg;base64,{encoded_image}"
            }
            }
        ]
        }
    ],
    "max_tokens": 300
    }

    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

    # Check if the response is successful
    if response.status_code == 200:
        print("here")
        # Parse the response as JSON
        answers = response.json()
    else:
        # Handle errors (non-200 responses)
        print(f"Error: {response.status_code} - {response.text}")

    return answers

In [7]:
image_number = 6

In [14]:
image_name = '../plot_images/'+str(image_number)+'.png'
answers = get_metadata_gpt(image_name)

here


In [15]:
answers

{'id': 'chatcmpl-8N20zSpwcIr4AezmXj7sYOMz91j64',
 'object': 'chat.completion',
 'created': 1700499981,
 'model': 'gpt-4-1106-vision-preview',
 'usage': {'prompt_tokens': 457,
  'completion_tokens': 292,
  'total_tokens': 749},
 'choices': [{'message': {'role': 'assistant',
    'content': 'Given that I cannot clearly read the details due to the image resolution, I can only provide rough estimations based on what can be interpreted from the provided image:\n\nanswers = {\n  "x-axis": {\n    "title": "Specific Capacity (mAh g^-1)",\n    "range": [0, 50],  # Estimation based on visible plot markings\n  },\n  "y-axis": {\n    "title": "Voltage (V)",\n    "range": [-0.2, 1.2],  # Estimation based on visible plot markings\n  },\n  "types": [\n    ("1st", "black"),  # Assuming the first line color corresponds to the \'1st\' cycle\n    ("2nd", "red"),    # Assuming the 2nd line color corresponds to the \'2nd\' cycle\n    ("10th", "green"),  # Assuming from the pattern that the \'10th\' cycle is